In [1]:
!pip install torch
!pip install transformers scikit-learn pandas numpy matplotlib seaborn

In [2]:
!pip install transformers

In [3]:
# Install required packages (run only if not already installed)
# !pip install torch transformers sklearn pandas numpy matplotlib seaborn

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    get_linear_schedule_with_warmup
)
# Import AdamW from optimization module
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import json
import warnings
from datetime import datetime
import os
import random
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
print(f"✅ Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

✅ PyTorch version: 2.8.0+cu126
✅ CUDA available: True
✅ Device: cuda


In [4]:
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import pandas as pd
import json
import re
import random
from datetime import datetime
from typing import List, Dict, Optional, Tuple, Any
import logging
import traceback
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Transformers and ML
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    pipeline
)

# Interface
import gradio as gr

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Device setup with error handling
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🚀 Device: {device}")
print(f"💾 CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💿 CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("\n✅ Setup complete!")

class MultiLanguageSupport:
    """Multi-language support system with translations"""

    def __init__(self):
        self.current_language = 'english'
        self.supported_languages = ['english', 'hindi', 'telugu', 'malayalam', 'kannada']

        # Language translations for UI elements
        self.translations = {
            'english': {
                'greeting': "Hello! I'm here to support your mental health. How can I help you today?",
                'assessment_menu': "Choose an assessment: 1 (PHQ-9), 2 (GAD-7), 3 (PCL-5)",
                'response_prompt': "Please respond with a number (0-{}) or describe your experience in your own words.",
                'interpretation': "I interpreted your response as: '{}' (option {})",
                'name_request': "Before we start the assessment, could you please tell me your name?",
                'email_request': "Please provide your email address to receive your results:",
                'age_request': "Thank you, {}! What is your age?",
                'sex_request': "What is your biological sex? (e.g., Male, Female, Other, Prefer not to say)",
                'marital_request': "What is your marital status? (e.g., Single, Married, Divorced, Widowed, Other)",
                'working_request': "What is your current working status? (e.g., Employed, Unemployed, Student, Retired, Other)",
                'crisis_message': "🚨 **IMMEDIATE CRISIS SUPPORT NEEDED**\n\nI'm very concerned about your safety. Please reach out for immediate help.",
                'help_menu': "## 🤖 How I Can Help You\n\nI'm here to provide mental health support in several ways:",
                'language_changed': "Language changed to English. You can continue our conversation.",
                'invalid_response': "Please enter a valid response.",
                'thank_you': "Thank you for providing your details. We will begin the assessment shortly.",
                'assessment_complete': "Thank you for completing the assessment. Your results will be reviewed by a doctor and sent to your email.",
                'login_prompt': "Please enter the doctor's password to view patient results.",
                'login_success': "Login successful. Here are the patient results.",
                'login_failed': "Incorrect password. Access denied.",
                'no_results': "No patient results available to display.",
                'email_sent': "Email sent successfully to the user.",
                'email_failed': "Failed to send email. Check configuration."
            },
            'hindi': {
                'greeting': "नमस्ते! मैं आपके मानसिक स्वास्थ्य का समर्थन करने के लिए यहाँ हूँ। आज मैं आपकी कैसे मदद कर सकता हूँ?",
                'assessment_menu': "एक मूल्यांकन चुनें: 1 (PHQ-9), 2 (GAD-7), 3 (PCL-5)",
                'response_prompt': "कृपया एक संख्या (0-{}) के साथ उत्तर दें या अपने अनुभव को अपने शब्दों में बताएं।",
                'interpretation': "मैंने आपके उत्तर की व्याख्या की है: '{}' (विकल्प {})",
                'name_request': "मूल्यांकन शुरू करने से पहले, कृपया मुझे अपना नाम बताएं?",
                'email_request': "अपने परिणाम प्राप्त करने के लिए कृपया अपना ईमेल पता प्रदान करें:",
                'age_request': "धन्यवाद, {}! आपकी उम्र क्या है?",
                'sex_request': "आपका जैविक लिंग क्या है? (जैसे, पुरुष, महिला, अन्य, बताना पसंद नहीं)",
                'marital_request': "आपकी वैवाहिक स्थिति क्या है? (जैसे, अविवाहित, विवाहित, तलाकशुदा, विधवा, अन्य)",
                'working_request': "आपकी वर्तमान कार्य स्थिति क्या है? (जैसे, कार्यरत, बेरोजगार, छात्र, सेवानिवृत्त, अन्य)",
                'crisis_message': "🚨 **तत्काल संकट सहायता की आवश्यकता**\n\nमैं आपकी सुरक्षा के बारे में बहुत चिंतित हूँ। कृपया तत्काल सहायता के लिए संपर्क करें।",
                'help_menu': "## 🤖 मैं आपकी कैसे मदद कर सकता हूँ\n\nमैं कई तरीकों से मानसिक स्वास्थ्य सहायता प्रदान करने के लिए यहाँ हूँ:",
                'language_changed': "भाषा हिंदी में बदल गई। आप हमारी बातचीत जारी रख सकते हैं।",
                'invalid_response': "कृपया एक मान्य उत्तर दर्ज करें।",
                'thank_you': "आपके विवरण प्रदान करने के लिए धन्यवाद। हम जल्द ही मूल्यांकन शुरू करेंगे।",
                'assessment_complete': "मूल्यांकन पूरा करने के लिए धन्यवाद। आपके परिणाम की डॉक्टर द्वारा समीक्षा की जाएगी और आपके ईमेल पर भेजे जाएंगे।",
                'login_prompt': "रोगी के परिणाम देखने के लिए कृपया डॉक्टर का पासवर्ड दर्ज करें।",
                'login_success': "लॉगिन सफल। यहाँ रोगी के परिणाम हैं।",
                'login_failed': "गलत पासवर्ड। पहुँच से वंचित।",
                'no_results': "कोई रोगी परिणाम उपलब्ध नहीं है।",
                'email_sent': "ईमेल उपयोगकर्ता को सफलतापूर्वक भेजा गया।",
                'email_failed': "ईमेल भेजने में विफल। कॉन्फ़िगरेशन जांचें।"
            },
            'telugu': {
                'greeting': "నమస్కారం! నేను మీ మానసిక ఆరోగ్యానికి మద్దతు ఇవ్వడానికి ఇక్కడ ఉన్నాను. ఈరోజు నేను మీకు ఎలా సహాయం చేయగలను?",
                'assessment_menu': "ఒక మూల్యాంకనాన్ని ఎంచుకోండి: 1 (PHQ-9), 2 (GAD-7), 3 (PCL-5)",
                'response_prompt': "దయచేసి ఒక సంఖ్యతో (0-{}) లేదా మీ అనుభవాన్ని మీ స్వంత మాటల్లో వివరించండి.",
                'interpretation': "నేను మీ ప్రతిస్పందనను ఇలా అర్థం చేసుకున్నాను: '{}' (ఎంపిక {})",
                'name_request': "మూల్యాంకనం మొదలుపెట్టడానికి ముందు, దయచేసి మీ పేరు చెప్పగలరా?",
                'email_request': "మీ ఫలితాలను పొందడానికి దయచేసి మీ ఈమెయిల్ చిరునామాను అందించండి:",
                'age_request': "ధన్యవాదాలు, {}! మీ వయస్సు ఎంత?",
                'sex_request': "మీ జీవసంబంధమైన లింగం ఏమిటి? (ఉదా, పురుషుడు, స్త్రీ, ఇతర, చెప్పడం ఇష్టం లేదు)",
                'marital_request': "మీ వైవాహిక స్థితి ఏమిటి? (ఉదా, అవివాహిత, వివాహిత, విడాకులు తీసుకున్న, వితంతు, ఇతర)",
                'working_request': "మీ ప్రస్తుత పని స్థితి ఏమిటి? (ఉదా, ఉద్యోగం, నిరుద్యోగం, విద్యార్థి, పదవీ విరమణ, ఇతర)",
                'crisis_message': "🚨 **తక్షణ సంక్షోభ సహాయం అవసరం**\n\nనేను మీ భద్రత గురించి చాలా ఆందోళన చెందుతున్నాను. దయచేసి తక్షణ సహాయం కోసం సంప్రదించండి.",
                'help_menu': "## 🤖 నేను మీకు ఎలా సహాయం చేయగలను\n\nనేను అనేక విధాలుగా మానసిక ఆరోగ్య మద్దతు అందించడానికి ఇక్కడ ఉన్నాను:",
                'language_changed': "భాష తెలుగుకు మార్చబడింది. మీరు మా సంభాషణ కొనసాగించవచ్చు.",
                'invalid_response': "దయచేసి చెల్లుబాటు అయ్యే ప్రతిస్పందన నమోదు చేయండి.",
                'thank_you': "మీ వివరాలు అందించినందుకు ధన్యవాదాలు. మేము త్వరలో మూల్యాంకనం ప్రారంభిస్తాము.",
                'assessment_complete': "మూల్యాంకనం పూర్తి చేసినందుకు ధన్యవాదాలు. మీ ఫలితాలను ఒక వైద్యుడు సమీక్షించి మీ ఈమెయిల్ కు పంపుతారు.",
                'login_prompt': "రోగి ఫలితాలను చూడటానికి దయచేసి డాక్టర్ పాస్‌వర్డ్‌ను నమోదు చేయండి.",
                'login_success': "లಾಗಿన్ విజయవంతమైంది. ఇక్కడ రోగి ఫలితాలు ఉన్నాయి.",
                'login_failed': "పాస్‌వర్డ్ తప్పు. యాక్సెస్ నిరాకరించబడింది.",
                'no_results': "ప్రదర్శించడానికి రోగి ఫలితాలు అందుబాటులో లేవు.",
                'email_sent': "ఈమెయిల్ విజయవంతంగా వినియోగదారుడికి పంపబడింది.",
                'email_failed': "ఈమెయిల్ పంపడంలో విఫലమైంది. కాన్ఫిగరేషన్ తనిఖీ చేయండి."
            },
            'malayalam': {
                'greeting': "നമസ്കാരം! നിങ്ങളുടെ മാനസികാരോഗ്യത്തെ പിന്തുണയ്ക്കാൻ ഞാനിവിടെയുണ്ട്. ഇന്ന് ഞാൻ നിങ്ങളെ എങ്ങനെ സഹായിക്കാൻ കഴിയും?",
                'assessment_menu': "ഒരു വിലയിരുത്തൽ തിരഞ്ഞെടുക്കുക: 1 (PHQ-9), 2 (GAD-7), 3 (PCL-5)",
                'response_prompt': "ദയവായി ഒരു സംഖ്യ (0-{}) ഉപയോഗിച്ച് അല്ലെങ്കിൽ നിങ്ങളുടെ അനുഭവം നിങ്ങളുടെ സ്വന്തം വാക്കുകളിൽ വിവരിക്കുക.",
                'interpretation': "ഞാൻ നിങ്ങളുടെ പ്രതികരണം ഇങ്ങനെ മനസ്സിലാക്കി: '{}' (ഓപ്ഷൻ {})",
                'name_request': "വിലയിരുത്തൽ ആരംഭിക്കുന്നതിന് മുമ്പ്, ദയവായി നിങ്ങളുടെ പേര് പറയാമോ?",
                'email_request': "നിങ്ങളുടെ ഫലങ്ങൾ ലഭിക്കാൻ ദയവായി നിങ്ങളുടെ ഇമെയിൽ വിലാസം നൽകുക:",
                'age_request': "നന്ദി, {}! നിങ്ങളുടെ പ്രായം എത്രയാണ്?",
                'sex_request': "നിങ്ങളുടെ ജൈവിക ലിംഗം എന്താണ്? (ഉദാ, പുരുഷൻ, സ്ത്രീ, മറ്റുള്ളവ, പറയാൻ ഇഷ്ടമില്ല)",
                'marital_request': "നിങ്ങളുടെ വിവാഹ നില എന്താണ്? (ഉദാ, അവിവാഹിത, വിവാഹിത, വിവാഹമോചിത, വിധവ, മറ്റുള്ളവ)",
                'working_request': "നിങ്ങളുടെ നിലവിലെ തൊഴിൽ നില എന്താണ്? (ഉദാ, ജോലിയുള്ള, തൊഴിലില്ലാത്ത, വിദ്യാർത്ഥി, വിരമിച്ച, മറ്റുള്ളവ)",
                'crisis_message': "🚨 **ഉടനടി പ്രതിസന്ധി സഹായം ആവശ്യം**\n\nനിങ്ങളുടെ സുരക്ഷയെക്കുറിച്ച് ഞാൻ വളരെ ആശങ്കാകുലനാണ്. ദയവായി ഉടനടി സഹായത്തിനായി ബന്ധപ്പെടുക.",
                'help_menu': "## 🤖 ഞാൻ നിങ്ങളെ എങ്ങനെ സഹായിക്കാൻ കഴിയും\n\nഞാൻ പല വഴികളിൽ മാനസികാരോഗ്യ പിന്തുണ നൽകാൻ ഇവിടെയുണ്ട്:",
                'language_changed': "ഭാഷ മലയാളത്തിലേക്ക് മാറി. നിങ്ങൾക്ക് ഞങ്ങളുടെ സംഭാഷണം തുടരാം.",
                'invalid_response': "ദയവായി സാധുവായ പ്രതികരണം നൽകുക.",
                'thank_you': "നിങ്ങളുടെ വിവരങ്ങൾ നൽകിയതിന് നന്ദി. ഞങ്ങൾ ഉടൻ തന്നെ വിലയിരുത്തൽ ആരംഭിക്കും.",
                'assessment_complete': "വിലയിരുത്തൽ പൂർത്തിയാക്കിയതിന് നന്ദി. നിങ്ങളുടെ ഫലങ്ങൾ ഒരു ഡോക്ടർ അവലോകനം ചെയ്യുകയും നിങ്ങളുടെ ഇമെയിലിലേക്ക് അയക്കുകയും ചെയ്യും.",
                'login_prompt': "രോഗിയുടെ ഫലങ്ങൾ കാണാൻ ദയവായി ഡോക്ടറുടെ പാസ്‌വേർഡ് നൽകുക.",
                'login_success': "ലോഗിൻ വിജയിച്ചു. ഇതാ രോഗിയുടെ ഫലങ്ങൾ.",
                'login_failed': "തെറ്റായ പാസ്‌വേഡ്. പ്രവേശനം നിഷേധിച്ചു.",
                'no_results': "പ്രദർശിപ്പിക്കാൻ രോഗിയുടെ ഫലങ്ങളൊന്നും ലഭ്യമല്ല.",
                'email_sent': "ഇമെയിൽ ഉപയോക്താവിന് വിജയകരമായി അയച്ചു.",
                'email_failed': "ഇമെയിൽ അയയ്ക്കാൻ കഴിഞ്ഞില്ല. കോൺഫിഗറേഷൻ പരിശോധിക്കുക."
            },
            'kannada': {
                'greeting': "ನಮಸ್ಕಾರ! ನಿಮ್ಮ ಮಾನಸಿಕ ಆರೋಗ್ಯವನ್ನು ಬೆಂಬಲಿಸಲು ನಾನು ಇಲ್ಲಿದ್ದೇನೆ. ಇಂದು ನಾನು ನಿಮಗೆ ಹೇಗೆ ಸಹಾಯ ಮಾಡಬಹುದು?",
                'assessment_menu': "ಮೌಲ್ಯಮಾಪನವನ್ನು ಆರಿಸಿ: 1 (PHQ-9), 2 (GAD-7), 3 (PCL-5)",
                'response_prompt': "ದಯವಿಟ್ಟು ಸಂಖ್ಯೆಯೊಂದಿಗೆ (0-{}) ಅಥವಾ ನಿಮ್ಮ ಅನುಭವವನ್ನು ನಿಮ್ಮ ಸ್ವಂತ ಮಾತುಗಳಲ್ಲಿ ವಿವರಿಸಿ.",
                'interpretation': "ನಾನು ನಿಮ್ಮ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ಹೀಗೆ ಅರ್ಥೈಸಿಕೊಂಡಿದ್ದೇನೆ: '{}' (ಆಯ್ಕೆ {})",
                'name_request': "ಮೌಲ್ಯಮಾಪನ ಪ್ರಾರಂಭಿಸುವ ಮೊದಲು, ದಯವಿಟ್ಟು ನಿಮ್ಮ ಹೆಸರನ್ನು ಹೇಳಬಹುದೇ?",
                'email_request': "ನಿಮ್ಮ ಫಲಿತಾಂಶಗಳನ್ನು ಪಡೆಯಲು ದಯವಿಟ್ಟು ನಿಮ್ಮ ಇಮೇಲ್ ವಿಳಾಸವನ್ನು ಒದಗಿಸಿ:",
                'age_request': "ಧನ್ಯವಾದಗಳು, {}! ನಿಮ್ಮ ವಯಸ್ಸು ಎಷ್ಟು?",
                'sex_request': "ನಿಮ್ಮ ಜೈವಿಕ ಲಿಂಗ ಯಾವುದು? (ಉದಾ, ಪುರುಷ, ಮಹಿಳೆ, ಇತರೆ, ಹೇಳಲು ಇಷ್ಟಪಡುವುದಿಲ್ಲ)",
                'marital_request': "ನಿಮ್ಮ ವೈವಾಹಿಕ ಸ್ಥಿತಿ ಯಾವುದು? (ಉದಾ, ಅವಿವಾಹಿತ, ವಿವಾಹಿತ, ವಿಚ್ಛೇದಿತ, ವಿಧವೆ, ಇತರೆ)",
                'working_request': "ನಿಮ್ಮ ಪ್ರಸ್ತುತ ಕೆಲಸದ ಸ್ಥಿತಿ ಯಾವುದು? (ಉದಾ, ಉದ್ಯೋಗ, ನಿರುದ್ಯೋಗ, ವಿದ್ಯಾರ್ಥಿ, ನಿವೃತ್ತ, ಇತರೆ)",
                'crisis_message': "🚨 **ತಕ್ಷಣ ಬಿಕ್ಕಟ್ಟು ಸಹಾಯದ ಅವಶ್ಯಕತೆ**\n\nನಿಮ್ಮ ಸುರಕ್ಷತೆಯ ಬಗ್ಗೆ ನಾನು ತುಂಬಾ ಚಿಂತಿತನಾಗಿದ್ದೇನೆ. ದಯವಿಟ್ಟು ತಕ್ಷಣ ಸಹಾಯಕ್ಕಾಗಿ ಸಂಪರ್ಕಿಸಿ.",
                'help_menu': "## 🤖 ನಾನು ನಿಮಗೆ ಹೇಗೆ ಸಹಾಯ ಮಾಡಬಹುದು\n\nನಾನು ಹಲವಾರು ವಿಧಾನಗಳಲ್ಲಿ ಮಾನಸಿಕ ಆರೋಗ್ಯ ಬೆಂಬಲವನ್ನು ಒದಗಿಸಲು ಇಲ್ಲಿದ್ದೇನೆ:",
                'language_changed': "ಭಾಷೆಯನ್ನು ಕನ್ನಡಕ್ಕೆ ಬದಲಾಯಿಸಲಾಗಿದೆ. ನೀವು ನಮ್ಮ ಸಂಭಾಷಣೆಯನ್ನು ಮುಂದುವರಿಸಬಹುದು。",
                'invalid_response': "ದಯವಿಟ್ಟು ಮಾನ್ಯವಾದ ಪ್ರತಿಕ್ರಿಯೆಯನ್ನು ನಮೂದಿಸಿ。",
                'thank_you': "ನಿಮ್ಮ ವಿವರಗಳನ್ನು ಒದಗಿಸಿದ್ದಕ್ಕಾಗಿ ಧನ್ಯವಾದಗಳು. ನಾವು ಶೀಘ್ರದಲ್ಲೇ ಮೌಲ್ಯಮಾಪನವನ್ನು ಪ್ರಾರಂಭಿಸುತ್ತೇವೆ。",
                'assessment_complete': "ಮೌಲ್ಯಮಾಪನವನ್ನು ಪೂರ್ಣಗೊಳಿಸಿದ್ದಕ್ಕಾಗಿ ಧನ್ಯವಾದಗಳು. ನಿಮ್ಮ ಫಲಿತಾಂಶಗಳನ್ನು ವೈದ್ಯರು ಪರಿಶೀಲಿಸಿ ನಿಮ್ಮ ಇಮೇಲ್‌ಗೆ ಕಳುಹಿಸಲಾಗುತ್ತದೆ。",
                'login_prompt': "ರೋಗಿಯ ಫಲಿತಾಂಶಗಳನ್ನು ನೋಡಲು ದಯವಿಟ್ಟು ವೈದ್ಯರ ಪಾಸ್‌ವರ್ಡ್ ಅನ್ನು ನಮೂದಿಸಿ。",
                'login_success': "ಲಾಗಿನ್ ಯಶಸ್ವಿಯಾಗಿದೆ. ಇಲ್ಲಿ ರೋಗಿಯ ಫಲಿತಾಂಶಗಳಿವೆ。",
                'login_failed': "ತಪ್ಪಾದ ಪಾಸ್‌ವರ್ಡ್. ಪ್ರವೇಶ ನಿರಾಕರಿಸಲಾಗಿದೆ。",
                'no_results': "ಪ್ರದರ್ಶಿಸಲು ಯಾವುದೇ ರೋಗಿ ಫಲಿತಾಂಶಗಳು ಲಭ್ಯವಿಲ್ಲ。",
                'email_sent': "ಇಮೇಲ್ ಯಶಸ್ವಿಯಾಗಿ ಬಳಕೆದಾರರಿಗೆ ಕಳುಹಿಸಲಾಗಿದೆ。",
                'email_failed': "ಇಮೇಲ್ ಕಳುಹಿಸಲು ವಿಫಲವಾಗಿದೆ. ಕಾನ್ಫಿಗರೇಶನ್ ಪರಿಶೀಲಿಸಿ。"
            }
        }

    def set_language(self, language: str) -> str:
        """Set the current language"""
        if language.lower() in self.supported_languages:
            self.current_language = language.lower()
            return self.get_text('language_changed')
        return f"Language '{language}' is not supported. Available languages: {', '.join(self.supported_languages)}"

    def get_text(self, key: str, *args) -> str:
        """Get translated text for current language"""
        text = self.translations[self.current_language].get(key, key)
        if args:
            return text.format(*args)
        return text

    def detect_language_change_request(self, message: str) -> Optional[str]:
        """Detect if user wants to change language"""
        message_lower = message.lower()

        language_keywords = {
            'english': ['english', 'en'],
            'hindi': ['hindi', 'हिंदी', 'hi'],
            'telugu': ['telugu', 'తెలుగు', 'te'],
            'malayalam': ['malayalam', 'മലയാളം', 'ml'],
            'kannada': ['kannada', 'ಕನ್ನಡ', 'kn']
        }

        if any(phrase in message_lower for phrase in ['change language', 'language change', 'switch language', 'भाषा बदले', 'భాష మార్చు', 'ഭാഷ മാറ്റുക', 'ಭಾಷೆ ಬದಲಿಸಿ']):
            for lang, keywords in language_keywords.items():
                if any(keyword in message_lower for keyword in keywords):
                    return lang

        # Direct language commands
        for lang, keywords in language_keywords.items():
            if message_lower in keywords:
                return lang

        return None

# Initialize multi-language support
language_support = MultiLanguageSupport()

class EnhancedResponseParser:
    """Enhanced response parser to handle both numeric and text responses"""

    def __init__(self):
        # Keyword mappings for different response types
        self.response_keywords = {
            # For 4-option questionnaires (PHQ-9, GAD-7)
            4: {
                0: ['not at all', 'never', 'no', 'none', 'bilkul nahi', 'nahi', 'లేదు', 'కుదరదు', 'ഇല്ല', 'ഇല്ല'],
                1: ['several days', 'sometimes', 'few days', 'kabhi kabhi', 'कभी कभी', 'కొన్ని రోజులు', 'ചില ദിവസങ്ങൾ', 'ಕೆಲವು ದಿನಗಳು'],
                2: ['more than half', 'most days', 'often', 'jyada din', 'ज्यादा दिन', 'ఎక్కువ రోజులు', 'കൂടുതൽ ദിവസങ്ങൾ', 'ಹೆಚ್ಚು ದಿನಗಳು'],
                3: ['nearly every day', 'always', 'daily', 'har din', 'हर दिन', 'ప్రతిరోజూ', 'എല്ലാ ദിവസവും', 'ಪ್ರತಿದಿನ']
            },
            # For 5-option questionnaires (PCL-5)
            5: {
                0: ['not at all', 'never', 'no', 'none', 'bilkul nahi', 'nahi', 'లేదు', 'ഇല്ല', 'ಇಲ್ಲ'],
                1: ['a little bit', 'slightly', 'thoda sa', 'थोड़ा सा', 'కొంచెం', 'അൽപം', 'ಸ್ವಲ್ಪ'],
                2: ['moderately', 'medium', 'beech mein', 'बीच में', 'మధ్యమంగా', 'മിതമായി', 'ಮಧ್ಯಮ'],
                3: ['quite a bit', 'much', 'kaafi', 'काफी', 'చాలా', 'വളരെ', 'ತುಂಬಾ'],
                4: ['extremely', 'very much', 'bahut jyada', 'बहुत ज्यादा', 'చాలా ఎక్కువ', 'വളരെ കൂടുതൽ', 'ಅತ್ಯಂತ']
            }
        }

    def parse_response(self, user_input: str, num_options: int) -> Tuple[int, str]:
        """Parse user response - returns (option_index, interpretation)"""
        user_input_clean = user_input.strip().lower()

        # Try to parse as direct number first
        try:
            response_index = int(user_input_clean)
            if 0 <= response_index < num_options:
                return response_index, f"Direct numeric response: {response_index}"
        except ValueError:
            pass

        # Use keyword matching based on number of options
        if num_options in self.response_keywords:
            keywords_map = self.response_keywords[num_options]

            # Calculate keyword scores for each option
            option_scores = {}
            for option_idx, keywords in keywords_map.items():
                score = 0
                for keyword in keywords:
                    if keyword in user_input_clean:
                        # Give higher score for longer keyword matches
                        score += len(keyword.split())
                option_scores[option_idx] = score

            # Find the option with highest score
            if option_scores and max(option_scores.values()) > 0:
                best_option = max(option_scores.items(), key=lambda x: x[1])[0]
                return best_option, f"Keyword matching: found relevant terms"

        # Fallback: analyze sentiment/intensity
        intensity_keywords = {
            'high': ['very', 'extremely', 'severe', 'intense', 'bahut', 'बहुत', 'చాలా', 'വളരെ', 'ತುಂಬಾ'],
            'medium': ['moderate', 'some', 'okay', 'thoda', 'थोड़ा', 'కొంచెం', 'അൽപം', 'ಸ್ವಲ್ಪ'],
            'low': ['little', 'slight', 'mild', 'kam', 'कम', 'తక్కువ', 'కుറവ്', 'ಕಡಿಮೆ'],
            'none': ['no', 'never', 'none', 'nahi', 'नहीं', 'లేదు', 'ഇല്ല', 'ಇಲ್ಲ']
        }

        for intensity, keywords in intensity_keywords.items():
            if any(keyword in user_input_clean for keyword in keywords):
                if intensity == 'none':
                    return 0, "Sentiment analysis: negative/none"
                elif intensity == 'low':
                    return 1 if num_options > 2 else 0, "Sentiment analysis: low intensity"
                elif intensity == 'medium':
                    return num_options // 2, "Sentiment analysis: medium intensity"
                elif intensity == 'high':
                    return num_options - 1, "Sentiment analysis: high intensity"

        # Ultimate fallback - return middle option
        return num_options // 2, "Default: middle option selected"

class MentalHealthQuestionnaires:
    """Complete questionnaire system with multi-language support"""

    def __init__(self):
        self.questionnaires = {
            'phq9': self._get_phq9(),
            'gad7': self._get_gad7(),
            'pcl5': self._get_pcl5()
        }
        print("✅ Mental Health Questionnaires initialized")

    def _get_phq9(self):
        return {
            'name': {
                'english': 'Patient Health Questionnaire-9 (PHQ-9)',
                'hindi': 'रोगी स्वास्थ्य प्रश्नावली-9 (PHQ-9)',
                'telugu': 'రోగి ఆరోగ్య ప్రశ్నావళి-9 (PHQ-9)',
                'malayalam': 'രോഗി ആരോഗ്യ ചോദ്യാവലി-9 (PHQ-9)',
                'kannada': 'ರೋಗಿ ಆರೋಗ್ಯ ಪ್ರಶ್ನಾವಳಿ-9 (PHQ-9)'
            },
            'description': {
                'english': 'Depression screening over the past 2 weeks',
                'hindi': 'पिछले 2 सप्ताह में अवसाद की जांच',
                'telugu': 'గత 2 వారాలలో డిప్రెషన్ పరీక్ష',
                'malayalam': 'കഴിഞ്ഞ 2 ആഴ്ചയിലെ വിഷാദം പരിശോധന',
                'kannada': 'ಕಳೆದ 2 ವಾರಗಳಲ್ಲಿ ಖಿന്നತೆಯ ಪರೀಕ್ಷೆ'
            },
            'questions': {
                'english': [
                    'Little interest or pleasure in doing things',
                    'Feeling down, depressed, or hopeless',
                    'Trouble falling or staying asleep, or sleeping too much',
                    'Feeling tired or having little energy',
                    'Poor appetite or overeating',
                    'Feeling bad about yourself or that you are a failure',
                    'Trouble concentrating on things',
                    'Moving or speaking slowly, or being fidgety/restless',
                    'Thoughts that you would be better off dead or hurting yourself'
                ],
                'hindi': [
                    'चीजों में कम रुचि या खुशी महसूस करना',
                    'उदास, अवसादग्रस्त या निराश महसूस करना',
                    'सोने में परेशानी या बहुत अधिक सोना',
                    'थका हुआ महसूस करना या कम ऊर्जा होना',
                    'भूख न लगना या अधिक खाना',
                    'अपने बारे में बुरा महसूस करना या असफल लगना',
                    'चीजों पर ध्यान देने में परेशानी',
                    'धीरे-धीरे बोलना या हिलना, या बेचैन रहना',
                    'यह सोचना कि मर जाना बेहतर होगा या खुद को नुकसान पहुंचाना'
                ],
                'telugu': [
                    'పనులలో తక్కువ ఆసక్తి లేదా ఆనందం అనుభవించడం',
                    'నిరాశగా, డిప్రెషన్‌లో లేదా నిస్సహాయంగా అనిపించడం',
                    'నిద్రలేకపోవడం లేదా ఎక్కువగా నిద్రపోవడం',
                    'అలసటగా అనిపించడం లేదా తక్కువ శక్తి ఉండటం',
                    'ఆకలి లేకపోవడం లేదా ఎక్కువగా తినడం',
                    'మీ గురించి చెడుగా అనిపించడం లేదా వైఫల్యంగా అనిపించడం',
                    'విషయాలపై దృష్టి పెట్టడంలో ఇబ్బంది',
                    'నెమ్మదిగా కదలడం లేదా మాట్లాడడం, లేదా చంచలంగా ఉండటం',
                    'మీరు చనిపోతే మంచిదని లేదా మీకు హాని చేసుకోవాలని అనిపించడం'
                ],
                'malayalam': [
                    'കാര്യങ്ങളിൽ കുറച്ച് താൽപ്പര്യം അല്ലെങ്കിൽ ആനന്ദം അനുഭവിക്കുക',
                    'നിരാശ, വിഷാദം, അല്ലെങ്കിൽ പ്രത്യാശയില്ലായ്മ അനുഭവിക്കുക',
                    'ഉറങ്ങാൻ ബുദ്ധിമുട്ട് അല്ലെങ്കിൽ കൂടുതൽ ഉറങ്ങുക',
                    'ക്ഷീണം അല്ലെങ്കിൽ കുറച്ച് ഊർജ്ജം അനുഭവിക്കുക',
                    'മോശം വിശപ്പ് അല്ലെങ്കിൽ അമിതഭക്ഷണം',
                    'നിങ്ങളെക്കുറിച്ച് മോശമായി അനുഭവിക്കുക അല്ലെങ്കിൽ പരാജയമായി തോന്നുക',
                    'കാര്യങ്ങളിൽ ശ്രദ്ധ കേന്ദ്രീകരിക്കാൻ പ്രയാസം',
                    'സാവധാനം നീങ്ങുക അല്ലെങ്കിൽ സംസാരിക്കുക, അല്ലെങ്കിൽ അസ്വസ്ഥത',
                    'നിങ്ങൾ മരിച്ചാൽ നല്ലതായിരിക്കും അല്ലെങ്കിൽ സ്വയം ദ്രോഹിക്കുക എന്ന ചിന്തകൾ'
                ],
                'kannada': [
                    'ವಿಷಯಗಳಲ್ಲಿ ಕಡಿಮೆ ಆಸಕ್ತಿ ಅಥವಾ ಸಂತೋಷವನ್ನು ಅನುಭವಿಸುವುದು',
                    'ನಿರಾಶೆ, ಖಿನ್ನತೆ, ಅಥವಾ ನಿರಾಶೆಯನ್ನು ಅನುಭವಿಸುವುದು',
                    'ನಿದ್ರೆಗೆ ತೊಂದರೆ ಅಥವಾ ಹೆಚ್ಚು ನಿದ್ರಿಸುವುದು',
                    'ಆಯಾಸ ಅಥವಾ ಕಡಿಮೆ ಶಕ್ತಿಯನ್ನು ಅನುಭವಿಸುವುದು',
                    'ಕಳಪೆ ಹಸಿವು ಅಥವಾ ಅತಿಯಾಗಿ ತಿನ್ನುವುದು',
                    'ನಿಮ್ಮ ಬಗ್ಗೆ ಕೆಟ್ಟದಾಗಿ ಭಾವಿಸುವುದು ಅಥವಾ ವಿಫಲತೆಯನ್ನು ಅನುಭವಿಸುವುದು',
                    'ವಿಷಯಗಳ ಮೇಲೆ ಗಮನ ಕೇಂದ್ರೀಕರಿಸುವಲ್ಲಿ ತೊಂದರೆ',
                    'ನಿಧಾನವಾಗಿ ಚಲಿಸುವುದು ಅಥವಾ ಮಾತನಾಡುವುದು, ಅಥವಾ ಚಂಚಲತೆ',
                    'ನೀವು ಸತ್ತರೆ ಉತ್ತಮ ಅಥವಾ ನಿಮಗೆ ಹಾನಿ ಮಾಡಿಕೊಳ್ಳುವ ಆಲೋಚನೆಗಳು'
                ]
            },
            'options': {
                'english': ['Not at all', 'Several days', 'More than half the days', 'Nearly every day'],
                'hindi': ['बिल्कुल नहीं', 'कुछ दिन', 'आधे से अधिक दिन', 'लगभग हर दिन'],
                'telugu': ['ఏ మాత్రం కాదు', 'కొన్ని రోజులు', 'సగానికి మించిన రోజులు', 'దాదాపు ప్రతిరోజూ'],
                'malayalam': ['ഒട്റ്റും ഇല്ല', 'കുറച്ച് ദിവസങ്ങൾ', 'പകുതിയിലധികം ദിവസങ്ങൾ', 'മിക്കവാറും എല്ലാ ദിവസവും'],
                'kannada': ['ಇಲ್ಲವೇ ಇಲ್ಲ', 'ಕೆಲವು ದಿನಗಳು', 'ಅರ್ಧಕ್ಕಿಂತ ಹೆಚ್ಚು ದಿನಗಳು', 'ಬಹುತೇಕ ಪ್ರತಿದಿನ']
            },
            'scoring': {
                (0, 4): 'Minimal depression',
                (5, 9): 'Mild depression',
                (10, 14): 'Moderate depression',
                (15, 19): 'Moderately severe depression',
                (20, 27): 'Severe depression'
            }
        }

    def _get_gad7(self):
        return {
            'name': {
                'english': 'Generalized Anxiety Disorder-7 (GAD-7)',
                'hindi': 'सामान्यीकृत चिंता विकार-7 (GAD-7)',
                'telugu': 'సాధారణ ఆందోళన రుగ్మత-7 (GAD-7)',
                'malayalam': 'സാധാരണവൽക്കരിച്ച ഉത്കണ്ഠാ വൈകല്യം-7 (GAD-7)',
                'kannada': 'ಸಾಮಾನ್ಯೀಕೃತ ಆತಂಕದ ಅಸ್ವಸ್ಥತೆ-7 (GAD-7)'
            },
            'description': {
                'english': 'Anxiety screening over the past 2 weeks',
                'hindi': 'पिछले 2 सप्ताह में चिंता की जांच',
                'telugu': 'గత 2 వారాలలో ఆందోళన పరీక్ష',
                'malayalam': 'കഴിഞ്ഞ 2 ആഴ്ചയിലെ ഉത്കണ്ഠ പരിശോധന',
                'kannada': 'ಕಳೆದ 2 ವಾರಗಳಲ್ಲಿ ಆತಂಕದ ಪರೀಕ್ಷೆ'
            },
            'questions': {
                'english': [
                    'Feeling nervous, anxious, or on edge',
                    'Not being able to stop or control worrying',
                    'Worrying too much about different things',
                    'Trouble relaxing',
                    'Being so restless that it is hard to sit still',
                    'Becoming easily annoyed or irritable',
                    'Feeling afraid, as if something awful might happen'
                ],
                'hindi': [
                    'घबराहट, चिंता या बेचैनी महसूस करना',
                    'चिंता को रोक या नियंत्रित न कर पाना',
                    'अलग-अलग चीजों के बारे में बहुत चिंता करना',
                    'आराम करने में परेशानी',
                    'इतना बेचैन होना कि एक जगह बैठना मुश्किल हो',
                    'आसानी से नाराज या चिड़चिड़ा हो जाना',
                    'डर लगना, जैसे कि कुछ भयानक हो सकता है'
                ],
                'telugu': [
                    'భయం, ఆందోళన లేదా అంచనాలలో ఉండటం',
                    'చింతను ఆపలేకపోవడం లేదా నియంత్రించలేకపోవడం',
                    'వివిధ విషయాల గురించి చాలా చింతించడం',
                    'విశ్రాంతి తీసుకోవడంలో ఇబ్బంది',
                    'ఒకే చోట కూర్చోలేనంత చంచలంగా ఉండటం',
                    'సులభంగా కోపం లేదా చిరాకు పడటం',
                    'భయం అనిపించడం, ఏదైనా భయంకరం జరుగుతుందని అనిపించడం'
                ],
                'malayalam': [
                    'പരിഭ്രമം, ഉത്കണ്ഠ, അല്ലെങ്കിൽ മുറുകെപ്പിടിക്കൽ അനുഭവിക്കുക',
                    'വേവലാതി നിർത്താനോ നിയന്ത്രിക്കാനോ കഴിയാതിരിക്കുക',
                    'വ്യത്യസ്ത കാര്യങ്ങളെക്കുറിച്ച് വളരെയധികം വേവലാതിപ്പെടുക',
                    'വിശ്രമിക്കാൻ പ്രയാസം',
                    'നിശ്ചലമായി ഇരിക്കാൻ പ്രയാസമാകുന്ന വിധം അസ്വസ്ഥത',
                    'എളുപ്പത്തിൽ ദേഷ്യപ്പെടുക അല്ലെങ്കിൽ പ്രകോപിതനാകുക',
                    'എന്തെങ്കിലും ഭയാനകമായ കാര്യം സംഭവിക്കുമെന്ന പോലെ ഭയം അനുഭവിക്കുക'
                ],
                'kannada': [
                    'ನರ, ಆತಂಕ, ಅಥವಾ ಅಂಚಿನಲ್ಲಿ ಇರುವ ಭಾವನೆ',
                    'ಚಿಂತೆಯನ್ನು ನಿಲ್ಲಿಸಲು ಅಥವಾ ನಿಯಂತ್ರಿಸಲು ಸಾಧ್ಯವಾಗದಿರುವುದು',
                    'ವಿವಿಧ ವಿಷಯಗಳ ಬಗ್ಗೆ ತುಂಬಾ ಚಿಂತಿಸುವುದು',
                    'ವಿಶ್ರಾಂತಿ ಪಡೆಯುವಲ್ಲಿ ಕಷ್ಟ',
                    'ಸ್ಥಿರವಾಗಿ ಕುಳಿತುಕೊಳ್ಳಲು ಕಷ್ಟವಾಗುವಷ್ಟು ಚಂಚಲವಾಗಿರುವುದು',
                    'ಸುಲಭವಾಗಿ ಕೋಪಗೊಳ್ಳುವುದು ಅಥವಾ ಕಿರಿಕಿರಿಯಾಗುವುದು',
                    'ಏನಾದರೂ ಭಯಾನಕ ಸಂಭವಿಸಬಹುದೆಂಬಂತೆ ಭಯ ಅನುಭವಿಸುವುದು'
                ]
            },
            'options': {
                'english': ['Not at all', 'Several days', 'More than half the days', 'Nearly every day'],
                'hindi': ['बिल्कुल नहीं', 'कुछ दिन', 'आधे से अधिक दिन', 'लगभग हर दिन'],
                'telugu': ['ఏ మాత్రం కాదు', 'కొన్ని రోజులు', 'సగానికి మించిన రోజులు', 'దాదాపు ప్రతిరోజೂ'],
                'malayalam': ['ഒട്റ്റും ഇല്ല', 'കുറച്ച് ദിവസങ്ങൾ', 'പകുതിയിലധികം ദിവസങ്ങൾ', 'മിക്കവാറും എല്ലാ ദിവസവും'],
                'kannada': ['ಇಲ್ಲವೇ ಇಲ್ಲ', 'ಕೆಲವು ದಿನಗಳು', 'ಅರ್ಧಕ್ಕಿಂತ ಹೆಚ್ಚು ದಿನಗಳು', 'ಬಹುತೇಕ ಪ್ರತಿದಿನ']
            },
            'scoring': {
                (0, 4): 'Minimal anxiety',
                (5, 9): 'Mild anxiety',
                (10, 14): 'Moderate anxiety',
                (15, 21): 'Severe anxiety'
            }
        }

    def _get_pcl5(self):
        return {
            'name': {
                'english': 'PTSD Checklist for DSM-5 (PCL-5)',
                'hindi': 'DSM-5 के लिए PTSD चेकलिस्ट (PCL-5)',
                'telugu': 'DSM-5 కోసం PTSD చెక్‌లిస్ట్ (PCL-5)',
                'malayalam': 'DSM-5-നുള്ള PTSD ചെക്ക്‌ലിസ്റ്റ് (PCL-5)',
                'kannada': 'DSM-5 ಗಾಗಿ PTSD ಪರಿಶೀಲನಾ ಪಟ್ಟಿ (PCL-5)'
            },
            'description': {
                'english': 'PTSD symptoms over the past month',
                'hindi': 'पिछले महीने में PTSD के लक्षण',
                'telugu': 'గత నెలలో PTSD లక్షణాలు',
                'malayalam': 'കഴിഞ്ഞ മാസത്തിലെ PTSD ലക്ഷണങ്ങൾ',
                'kannada': 'ಕಳೆದ ತಿಂಗಳಲ್ಲಿ PTSD ಲಕ್ಷಣಗಳು'
            },
            'questions': {
                'english': [
                    'Repeated, disturbing memories, thoughts, or images',
                    'Repeated, disturbing dreams of the stressful experience',
                    'Suddenly feeling or acting as if the event were happening again',
                    'Feeling very upset when something reminded you of the event',
                    'Physical reactions when something reminded you of the event'
                    # Shortened for brevity - you can add all 20 questions
                ],
                'hindi': [
                    'परेशान करने वाली यादें, विचार या छवियां बार-बार आना',
                    'तनावपूर्ण अनुभव के परेशान करने वाले सपने बार-बार आना',
                    'अचानक ऐसा लगना या व्यवहार करना जैसे घटना फिर हो रही हो',
                    'जब कोई चीज़ उस घटना की याद दिलाए तो बहुत परेशान हो जाना',
                    'जब कोई चीज़ उस घटना की याद दिलाए तो शारीरिक प्रतिक्रिया होना'
                ],
                'telugu': [
                    'పునరావృత, కలవరపరిచే జ్ఞాపకాలు, ఆలోచనలు లేదా చిత్రాలు',
                    'ఒత్తిడితో కూడిన అనుభవం యొక్క పునరావృత, కలవరపరిచే కలలు',
                    'అకస్మాత్తుగా సంఘటన మళ్లీ జరుగుతున్నట్లు అనిపించడం లేదా ప్రవర్తించడం',
                    'ఏదైనా సంఘటనను గుర్తుచేసినప్పుడు చాలా కలత చెందడం',
                    'ఏదైనా సంఘటనను గుర్తుచేసినప్పుడు శారీరక ప్రతిచర్యలు'
                ],
                'malayalam': [
                    'ആവർത്തിച്ചുള്ള, അസ്വസ്ഥപ്പെടുത്തുന്ന ഓർമ്മകൾ, ചിന്തകൾ, അല്ലെങ്കിൽ ചിത്രങ്ങൾ',
                    'സമ്മർദ്ദകരമായ അനുഭവത്തിന്റെ ആവർത്തിച്ചുള്ള, അസ്വസ്ഥപ്പെടുത്തുന്ന സ്വപ്നങ്ങൾ',
                    'സംഭവം വീണ്ടും നടക്കുന്നതുപോലെ പെട്ടെന്ന് തോന്നുകയോ പ്രവർത്തിക്കുകയോ ചെയ്യുക',
                    'എന്തെങ്കിലും സംഭവത്തെ ഓർമ്മിപ്പിച്ചപ്പോൾ വളരെ അസ്വസ്ഥത അനുഭവിക്കുക',
                    'എന്തെങ്കിലും സംഭവത്തെ ഓർമ്മിപ്പിച്ചപ്പോൾ ശാരീരിക പ്രതികരണങ്ങൾ'
                ],
                'kannada': [
                    'ಪುನರಾವರ್ತಿತ, ತೊಂದರೆಗೀಡುಮಾಡುವ ನೆನಪುಗಳು, ಆಲೋಚನೆಗಳು, ಅಥವಾ ಚಿತ್ರಗಳು',
                    'ಒತ್ತಡದ ಅನುಭವದ ಪುನರಾವರ್ತಿತ, ತೊಂದರೆಗೀಡುಮಾಡುವ ಕನಸುಗಳು',
                    'ಘಟನೆ ಮತ್ತೆ ನಡೆಯುತ್ತಿರುವಂತೆ ಇದ್ದಕ್ಕಿದ್ದಂತೆ ಅನಿಸಿಕೊಳ್ಳುವುದು ಅಥವಾ ವರ್ತಿಸುವುದು',
                    'ಏನಾದರೂ ಘಟನೆಯನ್ನು ನೆನಪಿಸಿದಾಗ ತುಂಬಾ ಅಸಮಾಧಾನ ಅನುಭವಿಸುವುದು',
                    'ಏನಾದರೂ ಘಟನೆಯನ್ನು ನೆನಪಿಸಿದಾಗ ದೈಹಿಕ ಪ್ರತಿಕ್ರಿಯೆಗಳು'
                ]
            },
            'options': {
                'english': ['Not at all', 'A little bit', 'Moderately', 'Quite a bit', 'Extremely'],
                'hindi': ['बिल्कुल नहीं', 'थोड़ा सा', 'मध्यम रूप से', 'काफी', 'अत्यधिक'],
                'telugu': ['ఏ మాత్రం కాదు', 'కొంచెం', 'మధ్యమంగా', 'చాలా', 'అధికంగా'],
                'malayalam': ['ഒട്റ്റും ഇല്ല', 'അൽപം', 'മിതമായി', 'വളരെ', 'അതിരുകടന്ന്'],
                'kannada': ['ಇಲ್ಲವೇ ಇಲ್ಲ', 'ಸ್ವಲ್ಪ', 'ಮಧ್ಯಮ', 'ತುಂಬಾ', 'ಅತ್ಯಂತ']
            }
            ,
            'scoring': {
                (0, 30): 'Minimal PTSD symptoms',
                (31, 37): 'Mild PTSD symptoms',
                (38, 49): 'Moderate PTSD symptoms',
                (50, 80): 'Severe PTSD symptoms'
            }
        }

    def get_severity(self, questionnaire_type: str, total_score: int) -> str:
        """Get severity level based on total score"""
        scoring = self.questionnaires[questionnaire_type]['scoring']
        for (min_score, max_score), severity in scoring.items():
            if min_score <= total_score <= max_score:
                return severity
        return 'Unknown'

    def get_questionnaire(self, qtype: str) -> Optional[Dict]:
        """Get questionnaire by type"""
        return self.questionnaires.get(qtype.lower())

class BERTProcessor:
    """Enhanced BERT processor for semantic understanding and classification"""

    def __init__(self, model_name: str = 'bert-base-uncased'):
        print(f"🔄 Loading BERT model: {model_name}")

        try:
            # Load tokenizer and model
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.classification_model = AutoModelForSequenceClassification.from_pretrained(
                model_name, num_labels=4
            )
            self.embedding_model = AutoModel.from_pretrained(model_name)

            # Move to device
            self.classification_model.to(device)
            self.embedding_model.to(device)

            # Set to eval mode
            self.classification_model.eval()
            self.embedding_model.eval()

            # Label mappings
            self.label2id = {'depression': 0, 'anxiety': 1, 'trauma': 2, 'general': 3}
            self.id2label = {v: k for k, v in self.label2id.items()}

            # Multi-language crisis keywords
            self.crisis_keywords = [
                # English
                'suicide', 'kill myself', 'end it all', 'better off dead',
                'hurt myself', 'self harm', 'cutting', 'overdose',
                'worthless', 'hopeless', "can't go on", 'no point',
                'want to die', 'thoughts of death', 'ending my life',
                'kill me', 'die', 'death wish',
                # Hindi
                'आत्महत्या', 'खुद को मार', 'सब खत्म कर', 'मरना बेहतर',
                'खुद को नुकसान', 'उम्मीद नहीं', 'जीने का मतलब नहीं',
                # Telugu
                'ఆత్మహత్య', 'చచ్చిపోవాలి', 'అంతం చేసుకోవాలి',
                # Malayalam
                'ആത്മഹത്യ', 'മരിക്കണം', 'അവസാനിപ്പിക്കണം',
                # Kannada
                'ಆತ್ಮಹತ್ಯೆ', 'ಸಾಯಬೇಕು', 'ಮುಗಿಸಬೇಕು'
            ]

            # Multi-language emotion keywords for fallback
            self.emotion_keywords = {
                'depression': [
                    'depressed', 'sad', 'hopeless', 'down', 'empty', 'numb', 'worthless',
                    'उदास', 'निराश', 'खाली', 'बेकार', 'నిరాశ', 'దుఖిత', 'వృధా',
                    'വിഷാദം', 'ദുഃഖം', 'നിരാശ', 'ಖಿನ್ನ', 'ದುಃಖ', 'ನಿರಾಶೆ'
                ],
                'anxiety': [
                    'anxious', 'worried', 'panic', 'nervous', 'scared', 'stressed', 'fear',
                    'चिंतित', 'घबरा', 'डर', 'तनाव', 'ఆందోళన', 'భయం', 'కలత',
                    'ഉത്കണ്ഠ', 'ഭയം', 'പരിഭ്രമം', 'ആതಂಕ', 'ಭಯ', 'ಒತ್ತಡ'
                ],
                'trauma': [
                    'trauma', 'ptsd', 'flashbacks', 'nightmares', 'triggered', 'abuse',
                    'आघात', 'दुर्व्यವಹಾರ', 'గాయం', 'వేధింపు', 'ആഘാതം', 'ദുരുപയോഗം',
                    'ಆಘಾತ', 'ನಿಂದನೆ'
                ]
            }

            print("✅ BERT processor loaded successfully.")
        except Exception as e:
            logger.error(f"Failed to load BERT models: {e}")
            self.tokenizer = None
            self.classification_model = None
            self.embedding_model = None
            print("❌ BERT model loading failed. Features requiring BERT will be unavailable.")

    def classify_text(self, text: str) -> str:
        """Classify a user message into a mental health category using BERT"""
        if not self.classification_model:
            return 'general'

        try:
            inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = self.classification_model(**inputs)

            # Get the predicted class
            predicted_class_id = outputs.logits.argmax().item()
            return self.id2label.get(predicted_class_id, 'general')
        except Exception as e:
            logger.error(f"BERT classification failed: {e}")
            return 'general'

    def check_for_crisis(self, text: str) -> bool:
        """Check for crisis keywords in the user's message"""
        text_lower = text.lower()
        return any(keyword in text_lower for keyword in self.crisis_keywords)

    def is_relevant_topic(self, text: str) -> bool:
        """Check if the text is relevant to mental health using keyword matching"""
        keywords = sum(self.emotion_keywords.values(), [])
        return any(keyword in text.lower() for keyword in keywords)


# --- Core Logic and State Management ---
class ChatState:
    """Manages the state of the conversation for each user session"""
    def __init__(self):
        self.state = 'initial'
        self.user_info = {}
        self.questionnaire_type = None
        self.question_index = 0
        self.scores = []
        self.all_responses = {}
        self.history = []

    def reset(self):
        """Resets the state for a new conversation"""
        self.state = 'initial'
        self.user_info = {}
        self.questionnaire_type = None
        self.question_index = 0
        self.scores = []
        self.all_responses = {}

class DoctorInterface:
    """Manages the doctor-facing interface and data"""
    def __init__(self, password: str):
        self.password = password
        self.patient_results = []
        self.logged_in = False
        self.selected_result_index = None
        print(f"Doctor password: {self.password}") # Added this line to print the password


    def add_result(self, result: Dict):
        """Adds a new patient result to the queue"""
        self.patient_results.append(result)

    def get_results_df(self):
        """Returns patient results as a pandas DataFrame for display"""
        if not self.patient_results:
            return pd.DataFrame()
        return pd.DataFrame(self.patient_results)

    def authenticate(self, input_password: str) -> bool:
        """Authenticates the doctor's password"""
        self.logged_in = (input_password == self.password)
        return self.logged_in

    def get_single_result(self, index: int) -> Optional[Dict]:
        """Returns a single patient result by index"""
        if 0 <= index < len(self.patient_results):
            return self.patient_results[index]
        return None

# --- Email Functionality ---
def send_email(to_email: str, subject: str, body: str):
    """Sends an email using a configured SMTP server"""
    # Replace with your SMTP server details
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    sender_email = "your_email@gmail.com"  # Your Gmail address
    sender_password = "your_app_password"  # Your App Password

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, to_email, msg.as_string())
        server.quit()
        logger.info(f"Email successfully sent to {to_email}")
        return True
    except Exception as e:
        logger.error(f"Error sending email to {to_email}: {e}")
        return False

# --- Global Instances ---
language_support = MultiLanguageSupport()
questionnaires = MentalHealthQuestionnaires()
response_parser = EnhancedResponseParser()
bert_processor = BERTProcessor()
doctor_interface = DoctorInterface(password="mysecretpassword123") # Set a strong password

# --- Gradio Bot Function ---
def chat_bot(message: str, history: List[Tuple[str, str]], state: Dict) -> Tuple[str, List[Tuple[str, str]], Dict]:
    try:
        # Restore session state from dictionary
        chat_state = ChatState()
        chat_state.__dict__ = state

        message_lower = message.lower().strip()
        history = history or []

        # 1. Handle Language Change Request
        lang_change_request = language_support.detect_language_change_request(message)
        if lang_change_request:
            response = language_support.set_language(lang_change_request)
            chat_state.state = 'initial' # Reset state after language change
            return "", history + [(message, response)], chat_state.__dict__

        # 2. Crisis Check (always on)
        if bert_processor.check_for_crisis(message):
            response = language_support.get_text('crisis_message')
            return "", history + [(message, response)], chat_state.__dict__

        # 3. Handle Assessment Flow
        if chat_state.state == 'initial':
            if message_lower in ['1', '2', '3']:
                q_map = {'1': 'phq9', '2': 'gad7', '3': 'pcl5'}
                chat_state.questionnaire_type = q_map.get(message_lower)
                chat_state.state = 'ask_name'
                response = language_support.get_text('name_request')
                return "", history + [(message, response)], chat_state.__dict__
            else:
                response = (
                    language_support.get_text('greeting') + "\n\n" +
                    language_support.get_text('assessment_menu') + "\n\n" +
                    language_support.get_text('help_menu')
                )
                return "", history + [(message, response)], chat_state.__dict__

        elif chat_state.state == 'ask_name':
            chat_state.user_info['name'] = message
            chat_state.state = 'ask_email'
            response = language_support.get_text('email_request')
            return "", history + [(message, response)], chat_state.__dict__

        elif chat_state.state == 'ask_email':
            chat_state.user_info['email'] = message
            chat_state.state = 'ask_age'
            response = language_support.get_text('age_request', chat_state.user_info['name'])
            return "", history + [(message, response)], chat_state.__dict__

        elif chat_state.state == 'ask_age':
            chat_state.user_info['age'] = message
            chat_state.state = 'ask_sex'
            response = language_support.get_text('sex_request')
            return "", history + [(message, response)], chat_state.__dict__

        elif chat_state.state == 'ask_sex':
            chat_state.user_info['sex'] = message
            chat_state.state = 'ask_marital'
            response = language_support.get_text('marital_request')
            return "", history + [(message, response)], chat_state.__dict__

        elif chat_state.state == 'ask_marital':
            chat_state.user_info['marital_status'] = message
            chat_state.state = 'ask_working'
            response = language_support.get_text('working_request')
            return "", history + [(message, response)], chat_state.__dict__

        elif chat_state.state == 'ask_working':
            chat_state.user_info['working_status'] = message
            chat_state.state = 'in_assessment'
            chat_state.question_index = 0
            response = language_support.get_text('thank_you')
            # Fallthrough to next state to ask first question immediately
            q = questionnaires.get_questionnaire(chat_state.questionnaire_type)
            question = q['questions'][language_support.current_language][chat_state.question_index]
            options = q['options'][language_support.current_language]
            response += f"\n\n**{q['name'][language_support.current_language]}**\n\nQ{chat_state.question_index + 1}: {question}\nOptions: {', '.join(options)}"

            return "", history + [(message, response)], chat_state.__dict__

        elif chat_state.state == 'in_assessment':
            q = questionnaires.get_questionnaire(chat_state.questionnaire_type)
            num_options = len(q['options']['english'])
            parsed_score, interpretation = response_parser.parse_response(message, num_options)

            chat_state.scores.append(parsed_score)
            chat_state.all_responses[f"Q{chat_state.question_index + 1}"] = {
                "question": q['questions'][language_support.current_language][chat_state.question_index],
                "response": message,
                "parsed_score": parsed_score
            }

            # Show interpretation to user
            interpretation_text = language_support.get_text('interpretation', interpretation, parsed_score)
            history.append((message, interpretation_text))

            chat_state.question_index += 1
            if chat_state.question_index < len(q['questions']['english']):
                next_question = q['questions'][language_support.current_language][chat_state.question_index]
                options = q['options'][language_support.current_language]
                response = f"Q{chat_state.question_index + 1}: {next_question}\nOptions: {', '.join(options)}"
            else:
                total_score = sum(chat_state.scores)
                severity = questionnaires.get_severity(chat_state.questionnaire_type, total_score)
                response = language_support.get_text('assessment_complete')

                # Store results for doctor
                result_data = {
                    'timestamp': datetime.now().isoformat(),
                    'user_info': chat_state.user_info,
                    'questionnaire_type': chat_state.questionnaire_type,
                    'total_score': total_score,
                    'severity': severity,
                    'responses': chat_state.all_responses
                }
                doctor_interface.add_result(result_data)

                # Send email to user (optional, can be done by doctor)
                # You can uncomment this if you want an automated email
                # email_body = f"Hello {chat_state.user_info['name']},\n\nThank you for completing the {q['name']['english']} assessment. Your results are:\n\nTotal Score: {total_score}\nSeverity: {severity}\n\nA doctor will review your results and provide a detailed analysis shortly.\n\nDisclaimer: This is not a substitute for professional medical advice."
                # send_email(chat_state.user_info['email'], "Mental Health Assessment Results", email_body)

                # Reset state
                chat_state.reset()

            return "", history + [(message, response)], chat_state.__dict__

        # 4. Handle general conversation (requires a conversational model, not implemented here)
        # This part of the code is commented out as it requires a large language model
        # elif not chat_state.state.startswith('in_assessment'):
        #     classification = bert_processor.classify_text(message)
        #     if classification == 'general':
        #         response = "I'm sorry, I am currently configured to assist with mental health assessments. Please select an assessment from the menu or change the language."
        #     else:
        #         # Here you would call a fine-tuned model for conversational support
        #         # For now, we'll just acknowledge the topic
        #         response = f"Thank you for sharing. I understand you're feeling {classification}. Would you like to take an assessment?"
        #     return history + [(message, response)], chat_state.__dict__

    except Exception as e:
        logger.error(f"Error in chat_bot: {e}\n{traceback.format_exc()}")
        response = "An error occurred. Let's restart. " + language_support.get_text('greeting')
        chat_state.reset()
        return "", history + [(message, response)], chat_state.__dict__

    return "", history, chat_state.__dict__


# --- Gradio Doctor Interface Function ---
def doctor_login(password: str):
    if doctor_interface.authenticate(password):
        return gr.update(visible=True), gr.update(visible=False), gr.update(value=language_support.get_text('login_success'))
    else:
        return gr.update(visible=False), gr.update(visible=True), gr.update(value=language_support.get_text('login_failed'))

def get_patient_data():
    df = doctor_interface.get_results_df()
    if df.empty:
        return gr.update(value=language_support.get_text('no_results')), gr.update(visible=False)

    df_display = df[['timestamp', 'user_info', 'questionnaire_type', 'total_score', 'severity']]
    return gr.update(value=df_display), gr.update(visible=True)

def show_detailed_results(evt: gr.SelectData):
    index = evt.index[0]
    result = doctor_interface.get_single_result(index)
    if not result:
        return "Result not found."

    doctor_interface.selected_result_index = index

    # Format the detailed results
    details_str = f"**Patient Information:**\n"
    for key, value in result['user_info'].items():
        details_str += f"- **{key.capitalize()}:** {value}\n"

    details_str += f"\n**Assessment:**\n"
    details_str += f"- **Type:** {result['questionnaire_type'].upper()}\n"
    details_str += f"- **Score:** {result['total_score']} ({result['severity']})\n"
    details_str += f"- **Timestamp:** {result['timestamp']}\n"

    details_str += f"\n**Detailed Responses:**\n"
    for q_num, data in result['responses'].items():
        details_str += f"- **{data['question']}**\n  > *User Response:* '{data['response']}' (Score: {data['parsed_score']})\n"

    return details_str

def send_doctor_analysis(user_analysis: str):
    if doctor_interface.selected_result_index is None:
        return "No patient selected to send analysis."

    result = doctor_interface.get_single_result(doctor_interface.selected_result_index)
    if not result:
        return "Patient data not found."

    to_email = result['user_info'].get('email')
    if not to_email:
        return "Patient email not available."

    subject = "Your Mental Health Assessment Analysis"
    body = f"Hello {result['user_info']['name']},\n\n"
    body += f"A doctor has reviewed your recent mental health assessment results. Here is their analysis:\n\n"
    body += f"**Doctor's Analysis:**\n{user_analysis}\n\n"
    body += f"**Your Assessment Summary:**\n"
    body += f"Type: {result['questionnaire_type'].upper()}\n"
    body += f"Score: {result['total_score']} ({result['severity']})\n"
    body += f"\nThank you for trusting us. Please seek professional medical help for further diagnosis and treatment.\n\n"
    body += "Sincerely,\nYour Mental Health Support Team"

    if send_email(to_email, subject, body):
        return language_support.get_text('email_sent')
    else:
        return language_support.get_text('email_failed')


# --- Gradio UI ---
def create_gradio_interface():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🤖 Mental Health Support Bot")

        # Main Chatbot Tab
        with gr.Tab("Chat with the Bot"):
            gr.Markdown("""
            ## Important Notes and Precautions
            * This chatbot is designed to provide initial mental health support and conduct standardized assessments (PHQ-9, GAD-7, PCL-5).
            * **It is NOT a substitute for professional medical advice, diagnosis, or treatment.** Always seek the advice of your physician or other qualified health provider with any questions you may have regarding a medical condition.
            * If you are in immediate crisis or danger, please reach out to a crisis hotline or emergency services. This bot is not equipped for emergency situations.
            * Your responses to the assessments will be stored securely for review by a medical professional via the Doctor Interface.

            ## How to Use the Bot
            1. **Start by saying hello or asking how the bot can help.**
            2. **Choose an assessment:** The bot will present you with options for PHQ-9, GAD-7, or PCL-5. Respond with the corresponding number (1, 2, or 3).
            3. **Provide your information:** The bot will ask for your name, age, sex, marital status, and working status. Please provide this information accurately.
            4. **Answer the assessment questions:** For each question, respond with a number corresponding to the option that best describes your experience over the specified time period (usually the past 2 weeks or month). You can also try describing your experience in your own words, and the bot will attempt to interpret it.
            5. **Review your results:** After completing the assessment, the bot will provide a summary of your score and severity level. Your detailed results will be available to the doctor for review.
            6. **Change Language:** You can change the language at any time by typing a phrase like "change language to Hindi" or simply typing the language name (e.g., "Telugu").
            7. **PHQ-9**(Depression screening).
            8. **GAD-7**(Anxiety screening).
            9. **PCL-5**(Trauma screening).
            """)
            chatbot = gr.Chatbot(label="Chat History")
            msg = gr.Textbox(placeholder="Type your message here...", label="Your Message")
            state = gr.State(value=ChatState().__dict__)

            # Use a dummy variable for the first message to trigger the bot
            initial_message = gr.Textbox(visible=False, value="start_chat")

            def initial_response(dummy, history, state):
                return chat_bot(language_support.get_text('greeting'), history, state)

            initial_message.change(fn=initial_response, inputs=[initial_message, chatbot, state], outputs=[msg, chatbot, state])

            msg.submit(
                fn=chat_bot,
                inputs=[msg, chatbot, state],
                outputs=[msg, chatbot, state]
            ).then(
                lambda: gr.update(value=''), inputs=None, outputs=[msg]
            )

        # Doctor Interface Tab
        with gr.Tab("Doctor Interface"):
            gr.Markdown("## Doctor Login")
            login_password = gr.Textbox(label=language_support.get_text('login_prompt'), type="password")
            login_button = gr.Button("Login")
            login_status = gr.Markdown()

            with gr.Column(visible=False) as doctor_dashboard:
                gr.Markdown("## Patient Assessment Results")
                refresh_button = gr.Button("Refresh Data")
                patient_data_df = gr.DataFrame(headers=["Timestamp", "Name", "Age", "Email", "Assessment", "Score", "Severity"], interactive=False)
                detailed_view = gr.Markdown("Click a row in the table to see detailed results.")

                with gr.Row():
                    analysis_textbox = gr.Textbox(label="Doctor's Analysis", placeholder="Enter your analysis here...")
                    send_email_button = gr.Button("Send Analysis to User via Email")

                email_status = gr.Markdown()

            login_button.click(
                fn=doctor_login,
                inputs=[login_password],
                outputs=[doctor_dashboard, login_button, login_status]
            ).then(
                fn=get_patient_data,
                inputs=None,
                outputs=[patient_data_df, login_button]
            )

            refresh_button.click(
                fn=get_patient_data,
                inputs=None,
                outputs=[patient_data_df, login_button]
            )

            patient_data_df.select(
                fn=show_detailed_results,
                outputs=[detailed_view]
            )

            send_email_button.click(
                fn=send_doctor_analysis,
                inputs=[analysis_textbox],
                outputs=[email_status]
            )

    return demo

if __name__ == '__main__':
    demo = create_gradio_interface()
    # To run on a public URL, set share=True
    # WARNING: This exposes your Gradio app and the doctor password field to the internet.
    # It is recommended to run this locally for security (share=False).
    demo.launch(share=False)

🔧 PyTorch version: 2.8.0+cu126
🚀 Device: cuda
💾 CUDA available: True
🎯 GPU: Tesla T4
💿 CUDA memory: 15.8 GB

✅ Setup complete!
✅ Mental Health Questionnaires initialized
🔄 Loading BERT model: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ BERT processor loaded successfully.
Doctor password: mysecretpassword123
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>